In [33]:
from gurobipy import *
import math

def distance(a,b):
  dx = a[0] - b[0]
  dy = a[1] - b[1]
  return math.sqrt(dx*dx + dy*dy)

# Problem data
clients = [[0, 1.5],[2.5, 1.2]]
facilities = [[0,0],[0,1],[0,1],
              [1,0],[1,1],[1,2],
              [2,0],[2,1],[2,2]]
charge = [3,2,3,1,3,3,4,3,2]

numFacilities = len(facilities)
numClients = len(clients)

m = Model()

# Add variables
x = {}
y = {}
d = {} # Distance matrix (not a variable)
alpha = 1

for j in range(numFacilities):
  x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)

for i in range(numClients):
  for j in range(numFacilities):
    y[(i,j)] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="t%d,%d" % (i,j))
    d[(i,j)] = distance(clients[i], facilities[j])

m.update()

# Add constraints
for i in range(numClients):
  for j in range(numFacilities):
    m.addConstr(y[(i,j)] <= x[j])

for i in range(numClients):
  m.addConstr(quicksum(y[(i,j)] for j in range(numFacilities)) == 1)

m.setObjective( quicksum(charge[j]*x[j] + quicksum(alpha*d[(i,j)]*y[(i,j)]
                for i in range(numClients)) for j in range(numFacilities)) )

m.optimize()

print(x)

Optimize a model with 20 rows, 27 columns and 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)

Root relaxation: objective 4.723713e+00, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.7237129    4.72371  0.00%     -    0s

Explored 0 nodes (15 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 4.72371 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.723712908962e+00, best bound 4.723712908962e+00, gap 0.0000%
{0: <gurobi.Var x0 (value -0.0)>, 1: <gurobi.Var x1 (value -

## ILP formulation of preference pair selection problem

\begin{align}
\max &  \sum_{j}{z_j} \\
	       s.t.  & \\
 		   & x_i \in \{ 0,1\} \\
           & z_j \in \{ 0,1\} \\
           & z_j\sum_{0 \leq i \leq n} x_i.P_{i,j} \geq 0 \\
           & 1 \leq j \leq m \,\, ,\,\, 1 \leq i \leq n
\end{align}



In [53]:
# Problem data
pmatrix = [[-0.2,-0.8, -0.12, -0.34, 0.98],
           [-0.1,-0.34, -0.24, -0.0001, 0.7],
           [-0.11,0.34, 0.1, -0.223, -0.34],
           [-0.45,0.04, -0.67, -0.23, -0.003],
           [-0.34,-0.31, 0.5, 0.8, 0.01]]

n = len(pmatrix)
m = 5

model = Model()

# Add variables
x = {}
z = {} 

for i in range(n):
  x[i] = model.addVar(vtype=GRB.BINARY, name="x%d" % i)

for j in range(m):
  z[j] = model.addVar(vtype=GRB.BINARY, name="z%d" % j)

model.update()

# Add constraints
for j in range(m):
    model.addGenConstrIndicator(z[j], False, quicksum(pmatrix[i][j]*x[i] for i in range(n)) <= 0.0)
    model.addGenConstrIndicator(z[j], True, quicksum(pmatrix[i][j]*x[i] for i in range(n)) >= 0.0001)
#     model.addConstr(quicksum(pmatrix[i][j]*x[i] for i in range(n))*z[j] >= 0)
#     model.addGenConstrIndicator(z[j], True, quicksum(pmatrix[i][j]*x[i] for i in range(n)), GRB.GREATER_EQUAL, 0.0)
#     model.addGenConstrIndicator(z[j], False, quicksum(pmatrix[i][j]*x[i] for i in range(n)), GRB., 0.0)



model.setObjective(quicksum(z[j] for j in range(m)), GRB.MAXIMIZE)
model.update()


model.optimize()

print(z)
print(x)

Optimize a model with 0 rows, 10 columns and 0 nonzeros
Model has 10 general constraints
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Presolve added 6 rows and 0 columns
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 6 rows, 8 columns, 35 nonzeros
Variable types: 0 continuous, 8 integer (8 binary)

Root relaxation: objective 3.790321e+00, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.79032    0    2   -0.00000    3.79032      -     -    0s
H    0     0                       3.0000000    3.79032  26.3%     -    0s

Explored 1 nodes (3 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available 